<a href="https://colab.research.google.com/github/annsun123/historical-code/blob/main/LSTM_(horizontal_TrainCut).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Model

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 11 13:31:44 2020
@author: anyan.sun

Steps:
  First: decide length(Steps) of the Xs: taking 75 percentile of counts that happen in past [5,10,15,20,25,30] days 
  Second: Prepare X_train, y_train
  Third: Train the Model and predict Output
  Fourth: Compare the result

"""

### Cutting Alternative I: 
# EEC, 1

import pandas as pd
import numpy as np
import datetime
from itertools import combinations

# there are 48, we are taing 
# which is i do have ac value which is greater than 
df_main=pd.read_excel('/content/drive/My Drive/Colab Notebooks/RR Folder/New_RR.xlsx') 
df_main['Date'] = df_main['Msg Time'].apply(lambda x: x.date())


## Decide length of X sequence


In [ ]:
df_RR = df_main[(df_main['Engine']==1) &(df_main['Source Type']=='EEC') 
        & (df_main['Side']==1.) & (df_main['FM'].notnull())]

date_opt=[5,10,15,20,25,30]

lenOpt_dic={}
for date in date_opt:
    len_opt = Get_Xlength(df_main, date, 'Date', duplicate='not_ignore')
    try:
        lenOpt_dic['date_rage: '+str(date)] = len_opt
    except:
        print('no value')

## AC_opt = df_RR.groupby('AC')['Date'].count()[df_RR.groupby('AC')['Date'].count()>1000].keys()

df_RR['FM'] = df_RR['FM'].astype('category')
df_RR['fm_cat'] = df_RR['FM'].cat.codes

NameError: ignored

In [ ]:
lenOpt_dic

{'date_rage: 10': 17.0,
 'date_rage: 15': 24.0,
 'date_rage: 20': 29.0,
 'date_rage: 25': 31.75,
 'date_rage: 30': 39.0,
 'date_rage: 5': 10.0}

In [ ]:
cardinality = len(df_RR['fm_cat'].unique())
seq_length = int(lenOpt_dic['date_rage: 15'])
predict_gap = 5
batch_size = 5

In [ ]:
CatFM_dic={}
FMCat_dic={}
for ind, val in df_RR.drop_duplicates(['fm_cat','FM'])[['fm_cat','FM']].iterrows():
  CatFM_dic[val['fm_cat']]=val['FM']
  FMCat_dic[val['FM']]=val['fm_cat']


## Dropping the duplicated values 

In [ ]:
df=pd.DataFrame()
for ac in df_RR['AC'].unique():
    df_ac=df_RR[df_RR['AC']==ac].\
    sort_values(['Msg Time'])
    
    ##df_ac['class_1'] = df_ac['msg_text'].apply(lambda x: x.split('-')[0].strip())
    #df_ac['engine_class'] = df_ac[['class_1', 'Source']].apply(lambda x: x['class_1']+ '_'+x['Source'],1)
   # df1=df_check.drop_duplicates(['engine_class','FM']).groupby('engine_class')['FM'].count()
    x=0
    del_lst=[]
    while x<len(df_ac)-1:    
        if df_ac['Fault Code'].tolist()[x+1]==df_ac['Fault Code'].tolist()[x]:
            del_lst.append(x)
        x+=1
       
    df_ac=df_ac.drop(df_ac.index[del_lst]).reset_index(drop=True)
    df=df.append(df_ac)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Merging Small AC into 

remove the ac that is below certain number that is less than 25% 
  * merging the ac less 25%

In [ ]:
# minor_ac = np.percentile(df_RR.groupby('AC')['Date'].count(),25)
minor_ac = seq_length+ (2*batch_size)
AC_opt = df_RR.groupby('AC')['Date'].count()[df_RR.groupby('AC')['Date'].count()>=minor_ac].keys()
AC_leftOut = df_RR.groupby('AC')['Date'].count()[df_RR.groupby('AC')['Date'].count()<minor_ac].keys()

In [ ]:
df_normalAC=df_RR[df_RR['AC'].isin(AC_opt)]

In [ ]:
df_minor=pd.DataFrame()
for ind, ac in enumerate(AC_leftOut):
  df_ac=df_RR[df_RR['AC']==ac]
  df_minor=df_minor.append(df_ac)

df_minor['AC']='merge_'+str(ind)
df_normalAC=df_normalAC.append(df_minor)


In [ ]:
df_normalAC.groupby('AC')['Date'].count()

AC
1           376
2           382
3           506
4           383
5           387
6           408
7           171
8           486
9           459
10          501
11          376
12          111
13          239
14          164
15          191
16          244
17          214
18          106
19          268
21           38
22          161
23           80
25           52
34          104
35          154
36           82
merge_10    110
Name: Date, dtype: int64

## FIltering out Fault Code that is less than 5% 
** take fault count for every air craft, then remove fault count below 5% (consider the faults that only happen in nature)


In [ ]:
from collections import Counter
df_final = pd.DataFrame()
for ac in df_normalAC['AC'].unique():
  df_ac = df_normalAC[df_normalAC['AC']==ac]
  minor_value=np.percentile(list(Counter(df_ac['FM'].tolist()).values()),5)
  fm_lst=df_ac.groupby('FM')['FM'].count()[df_ac.groupby('FM')['FM'].count()>minor_value].index.tolist()
  df_ac=df_ac[df_ac['FM'].isin(fm_lst)]
  df_final = df_final.append(df_ac)

In [ ]:
print(len(df_final))
print(len(df_normalAC))

6667
6753


In [ ]:
score_log={}

## Test with last 20 %

### Validation Method I:

In [ ]:
def gen_80Test(df_RR, rolling_step, AC_opt, cardinality, seq_length, predict_gap, batch_size):
  X_train=[]
  y_train=[]
  X_test=[]
  y_test=[]

  min_len = seq_length + predict_gap
  df_train=pd.DataFrame(columns=df_RR['fm_cat'].unique())
  for ac in AC_opt:
      df_ac = df_RR[df_RR['AC']==ac].sort_values('Date')
      ac_lst=df_ac['fm_cat'].tolist()[:int(len(df_ac)*0.8)]
      sub_X, sub_y = generate_sequence(ac_lst, seq_length, predict_gap, rolling_step)
      X_train.extend(sub_X)
      y_train.extend(sub_y)
     
      
      if len(ac_lst)*0.2 > min_len+seq_length:
          test_lst=ac_lst[-int(len(ac_lst)*0.2):]
      else: 
          test_lst=ac_lst[-(min_len+seq_length):]
      sub_X, sub_y = generate_sequence(test_lst, seq_length, predict_gap, rolling_step)
      X_test.extend(sub_X)
      y_test.extend(sub_y)
  X_train = to_categorical(X_train, num_classes=cardinality)
  y_train = to_categorical(y_train, num_classes=cardinality)


  X_test = to_categorical(X_test, num_classes=cardinality)
  y_test = to_categorical(y_test, num_classes=cardinality)

    
  return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = \
    gen_80Test(df_final,1, df_final['AC'].unique().tolist(),  cardinality, seq_length, predict_gap, batch_size)  

X_train = X_train[len(X_train) % batch_size:]
y_train = y_train[len(y_train) % batch_size:]
X_test = X_test[len(X_test) % batch_size:]
y_test = y_test[len(y_test) % batch_size:]

#encoded_length = np.array(X_train).shape[2]
#output = model_predict(seq_length, predict_gap, 
 #             encoded_length,
  #            batch_size, np.array(X_train),
   #           np.array(y_train), np.array(X_test))

#true_true, false_false = pred_score(output, y_test)



In [ ]:
print(true_true, false_false)

0.9508771929824561 0.07894736842105263


In [ ]:
overall_8_2=[true_true, false_false]
score_log['overall_8_2']=overall_8_2

In [ ]:
acc_dic={}
fakse_dic={}
for ac in df_final['AC'].unique().tolist():

  try:
    X_train, y_train, X_test, y_test = \
      gen_80Test(df_final,1, [ac],  cardinality, seq_length, predict_gap, batch_size)


    encoded_length = np.array(X_train).shape[2]
    output = model_predict(seq_length, predict_gap, 
                  encoded_length,
                  batch_size, np.array(X_train),
                  np.array(y_train), np.array(X_test))
    true_true, false_false = pred_score(output, y_test)
    acc_dic[ac]=true_true
    fakse_dic[ac]=false_false
  except:
    print('check array of ac_'+str(ac))

Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20

In [ ]:
score_log

In [ ]:
score_log['individual_ac']=[acc_dic,fakse_dic]

### Validation Method II: 

In [ ]:
encoded_length = np.array(X_train).shape[2]

model = model_fit(seq_length, predict_gap, 
            encoded_length,
            batch_size, np.array(X_train),
            np.array(y_train))



Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20


In [ ]:
np.array(testX).shape

(5, 24, 49)

In [ ]:
X_train = X_train[len(X_train) % batch_size:]
y_train = y_train[len(y_train) % batch_size:]
X_test = X_test[len(X_test) % batch_size:]
y_test = y_test[len(y_test) % batch_size:]

In [ ]:
validation_scr = {}

In [ ]:
 X_train, y_train, X_test, y_test = \
      gen_80Test(df_final, 5, [ac], cardinality, seq_length, predict_gap, batch_size)

In [ ]:
# actually is making rolling windows with 5 steps
validationII_agg = []
for roll_step in range(1,4,1):
  validation_scr={}
  validation_scr['ind_defi']=['length_of_y', 'true_true', 'false_false']
  for ac in df_final['AC'].unique():

    try:
      X_train, y_train, X_test, y_test = \
            gen_80Test(df_final, roll_step, [ac], cardinality, seq_length, predict_gap, batch_size)
      initial_test = X_test[0]
      testX = [initial_test for n in range(5)]
        
      ac_score=[]
      ac_FTscore=[]
      
    
      for y in range(len(y_test)):
        
        y_true = [np.argmax(y_test[y][i]) for i in range(5)]
        output = model.predict(np.array(testX))[0]
      # for validation method 2, how many steps that we are going to loop over
        pred_FM = []
        for i in range(len(output)):
          pred_FM.append(np.argmax(output[i]))
        ac_score.append(len(set(pred_FM) & set(y_true))/len(set(y_true)))
        ac_FTscore.append(1-(len(set(pred_FM) & set(y_true))/len(set(pred_FM))))
        testArray = testX[0][roll_step:]
        testX = np.append(testArray,output[:roll_step]).reshape(seq_length,cardinality)
        testX=[testX for n in range(5)]
      validation_scr['AC_'+str(ac)]=[len(y_test), np.sum(ac_score)/len(ac_score),np.sum(ac_FTscore)/len(ac_FTscore) ]
   
    except:
      print('check length of df_ac')
  validationII_agg.append(validation_scr)
 

check length of df_ac
check length of df_ac
check length of df_ac


In [ ]:
validation_scr={}

{'AC_1': [4, 0.5, 0.25],
 'AC_10': [8, 1.0, 0.0],
 'AC_11': [4, 0.5833333333333334, 0.25],
 'AC_12': [3, 0.8333333333333334, 0.0],
 'AC_13': [3, 0.0, 1.0],
 'AC_14': [3, 1.0, 0.0],
 'AC_15': [3, 0.6666666666666666, 0.0],
 'AC_16': [3, 0.0, 1.0],
 'AC_17': [3, 0.38888888888888884, 0.0],
 'AC_18': [3, 0.8333333333333334, 0.16666666666666666],
 'AC_19': [3, 0.16666666666666666, 0.6666666666666666],
 'AC_2': [5, 1.0, 0.0],
 'AC_22': [3, 0.16666666666666666, 0.6666666666666666],
 'AC_23': [3, 0.8333333333333334, 0.0],
 'AC_3': [8, 1.0, 0.0],
 'AC_34': [3, 1.0, 0.0],
 'AC_35': [3, 1.0, 0.0],
 'AC_36': [3, 1.0, 0.0],
 'AC_4': [5, 1.0, 0.0],
 'AC_5': [5, 0.0, 1.0],
 'AC_6': [5, 1.0, 0.0],
 'AC_7': [3, 0.8333333333333334, 0.0],
 'AC_8': [8, 0.9375, 0.0],
 'AC_9': [7, 1.0, 0.0],
 'AC_merge_10': [3, 0.38888888888888884, 0.16666666666666666],
 'ind_defi': ['length_of_y', 'true_true', 'false_false']}

In [ ]:
# overall score
for roll_step in range(len(validationII_agg)):
  validation_scrSEE=validationII_agg[roll_step]
  print('the overall scr for roll_step_' + str(roll_step) +
        ' is '+str(np.sum([validation_scrSEE[key][1] for key in list(validation_scrSEE.keys())[1:]])/len(list(validation_scrSEE.keys())[1:])))
  print("""there are """ + 
        str(len([validation_scrSEE[key][1] for key in list(validation_scrSEE.keys())[1:] \
                 if validation_scrSEE[key][1]>0.75])) +
  """ above 0.75 and there are total """ +
   str(len(df_final['AC'].unique())) + ' which is ' +str(14/27))
 
# print(np.sum(validation_scr[key][1])/len(list(validation_scr.keys())[1:])

the overall scr for roll_step_0 is 0.6162913173782738
there are 14 above 0.75 and there are total 27 which is 0.5185185185185185
0.20346320346320346
the overall scr for roll_step_1 is 0.6776537564999102
there are 14 above 0.75 and there are total 27 which is 0.5185185185185185
0.336431623931624
the overall scr for roll_step_2 is 0.6519383394383395
there are 14 above 0.75 and there are total 27 which is 0.5185185185185185
0.27662037037037035


In [ ]:
print(np.sum([validation_scrSEE[key][1] for key in list(validation_scrSEE.keys())[1:] \
                 if validation_scrSEE[key][1]<0.75])/len([validation_scrSEE[key][1] for key in list(validation_scrSEE.keys())[1:] \
                 if validation_scrSEE[key][1]<0.75]))
  

In [ ]:
import json
with open('validationIIAGG.json', 'w') as file:
  json.dump(validationII_agg, file)

In [ ]:
X_train, y_train, X_test, y_test = \
          gen_80Test(df_final, 5, df_final['AC'].unique().tolist(), cardinality, seq_length, predict_gap, batch_size)
initial_test = X_test[0]
testX = [initial_test for n in range(5)]

ac_score=[]


for y in range(len(y_test)):
  y_true = [np.argmax(y_test[y][i]) for i in range(5)]
  output = model.predict(np.array(testX))[0]
  pred_FM=[]
# for validation method 2, how many steps that we are going to loop over
  for i in range(len(output)):
    pred_FM.append(np.argmax(output[i]))
  ac_score.append(len(set(pred_FM) & set(y_true))/len(set(y_true)))

  testArray = testX[0][5:]
  testX = np.append(testArray,output).reshape(seq_length,cardinality)
  testX=[testX for n in range(5)]
#validation_scr[str(ac)+'scr_validationII']=
print(np.sum(ac_score)/len(ac_score))

0.09615384615384616


In [ ]:
validation_scr['overall_scr_validationII']=np.sum(overall_score)/len(overall_score)

In [ ]:
validation_scr

{'overall_scr_validationII': 0.25833333333333336}

In [ ]:
testX=[initial_test for n in range(5)]
output = model.predict(np.array([testArray_new for n in range(5)]))[0]
for i in output:
  print(np.argmax(i))

40
40
40
40
40


In [ ]:
for i in y_test[2]:
  print(np.argmax(i))

40
40
40
40
40


## Test with last three months


In [ ]:
# to split the data into those before 3 months and those after 3 months
# so for example, there are mltiple ac, and there will be i 

In [ ]:
acc3Month_dic={}
false3Month_dic={}

for ac in AC_opt:
  try:
    TrainSplit_Cut='vertical'
    X_train, y_train, X_test, y_test = \
      gen_TrainTest(df_final, TrainSplit_Cut,'3_month',1, df_final['AC'].unique().tolist(),  cardinality, seq_length, predict_gap, batch_size)


    encoded_length = np.array(X_train).shape[2]
    output = model_predict(seq_length, predict_gap, 
                  encoded_length,
                  batch_size, np.array(X_train),
                  np.array(y_train), np.array(X_test))
    true_true, false_false = pred_score(output, y_test)
    acc3Month_dic[ac]=true_true
    false3Month_dic[ac]=false_false

  except:
    print('check array of ac_'+str(ac))

Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
check array of ac_2
Epoch 1/20
Epoch 2/20
check array of ac_3
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
check array of ac_4
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoc

In [ ]:
acc3Month_dic

NameError: ignored

In [ ]:
true_true, false_false = pred_score(output, y_test)
true_true

0.8894736842105263

## Alternative I: testing with 3 rolling steps
taking subset of last three month---doing the rolling window with more stpes (more than 1), like 3,4,5


In [ ]:
TrainSplit_Cut='vertical'
X_train, y_train, X_test, y_test = \
    gen_TrainTest(df_final, TrainSplit_Cut, '20_per', 3, df_final['AC'].unique().tolist(),  cardinality, seq_length, predict_gap, batch_size)
   

encoded_length = np.array(X_train).shape[2]
output = model_predict(seq_length, predict_gap, 
              encoded_length,
              batch_size, np.array(X_train),
              np.array(y_train), np.array(X_test))
true_true, false_false = pred_score(output, y_test)


Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20


In [ ]:
score_log['rolling_step_3']=[true_true, false_false]

In [ ]:
with open ('score_log_new.json', 'w') as file:
  json.dump(score_log,file)

# Predicting the Future 


and return one by one




Step One: combine all the small AC as one
	report accuracy [80%, 20% ] # taking the last 20 out.
take fault count for every air craft, then remove fault count below 5% (consider the faults that only happen in nature)
---if found out total 60% records re-occurance, so the accuracy is always 60%. So only use some of the fault code. 
 	then recreate the datasets

Step Three: 

keep 25 sequence as it is, then next sequence for ac_1. The array will be 1000 (len of list 20, )
1000, 25 each and 4 rows. For all the ac 
all these will be category. 


* how many length and features we are keeping. backfill or forwardfill with previous data

	start with ac=1 and then create 5 rows. Then again come back
	take flight phase_2 and take all aircraft then put into sequence of fault that happen

 


train with different windows, and using that window and train with 100% and take last piece and predict the future coming result
dynamic moving windows for different AC 
5 different Moving windows to test with AC and pickk up the best moving window which return best result.


ac_lst = df_RR[df_RR['AC']==2]['FM'].tolist()

## Predicting Next Coming Steps

In [ ]:
X_final=[]
y_final=[]
min_len = seq_length + (2*batch_size)

for ac in AC_opt:#
    df_ac = df_final[df_final['AC']==ac].sort_values('Date')
    ac_lst=df_ac['fm_cat'].tolist()[:int(len(df_ac))]
    sub_X, sub_y = generate_sequence(ac_lst, seq_length, predict_gap, 1)
    X_final.extend(sub_X)
    y_final.extend(sub_y)

X_final = to_categorical(X_train, num_classes=cardinality)
y_final = to_categorical(y_train, num_classes=cardinality)
X_final = X_train[len(X_train) % batch_size:]
y_final = y_train[len(y_train) % batch_size:]


In [ ]:
model = model_fit(seq_length, predict_gap, 
            encoded_length,
            batch_size, np.array(X_final),
            np.array(y_final))


Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20


### Last Step Data

In [ ]:
min_len = seq_length + predict_gap
ac = 3
ac_lst = df_final[df_final['AC']==ac].sort_values('Date')['fm_cat'].tolist()


### Doing Rolling windows for ten 

In [ ]:
final_lst = ac_lst[-(10 + seq_length):]
last_X = []
for step in range(0, len(final_lst)-seq_length):
  last_X.append(final_lst[step:step+seq_length])
 # final_lst.append(final_lst[step:step+seq_length])
  
final_array = to_categorical(last_X, num_classes=cardinality)
encoded_length = np.array(final_array).shape[2]
output=model.predict(final_array, batch_size=batch_size, verbose=0)


In [ ]:
final_output=[]
for i in output:
  pred_seq = []
  for step in range(predict_gap):
    pred_seq.append(CatFM_dic[np.argmax(i[step])])
  final_output.append(pred_seq)

In [ ]:
final_output

[[458.0, 458.0, 458.0, 458.0, 458.0],
 [460.0, 460.0, 460.0, 458.0, 458.0],
 [458.0, 458.0, 458.0, 458.0, 458.0],
 [223.0, 458.0, 458.0, 458.0, 458.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [458.0, 458.0, 458.0, 458.0, 458.0],
 [458.0, 458.0, 458.0, 458.0, 458.0],
 [458.0, 458.0, 458.0, 458.0, 458.0],
 [458.0, 458.0, 458.0, 458.0, 458.0],
 [458.0, 460.0, 460.0, 460.0, 460.0]]

### Last Step without Rolling Windows

In [ ]:
# min_len = seq_length + predict_gap
#for ac in [1,2,3]:
last_X = []
for ac in df_final['AC'].unique():
  ac_lst = df_final[df_final['AC']==ac].sort_values('Date')['fm_cat'].tolist()

  last_X.append(ac_lst[-seq_length:])
  # final_lst.append(final_lst[step:step+seq_length])
final_array = to_categorical(last_X, num_classes=cardinality)
encoded_length = np.array(final_array).shape[2]
output=model.predict(final_array, batch_size=batch_size, verbose=0)


In [ ]:
final_output=[]
for i in output:
  pred_seq = []
  for step in range(predict_gap):
    pred_seq.append(CatFM_dic[np.argmax(i[step])])
  final_output.append(pred_seq)

In [ ]:
final_output[2]

[458.0, 458.0, 458.0, 458.0, 458.0]

## Prediction with random choice

[The performance is quiet nice]

In [ ]:
random_sample = []
random_output = []
min_len = seq_length + predict_gap

#for ac in df_final['AC'].unique().tolist():
ac = 2
ac_lst = df_final[df_final['AC']==ac].sort_values('Date')['fm_cat'].tolist()

if len(ac_lst)*0.2 > 2*seq_length:
    test_lst = ac_lst[-int(len(ac_lst)*0.2):]
else: 
    test_lst = ac_lst[-(2*seq_length):]

random_lst=[]

for n in range(50):
  sample = random.sample(test_lst,seq_length)
  random_lst.append(sample)

sample_array = to_categorical(random_lst, num_classes=cardinality)
encoded_length = np.array(X_train).shape[2]
output=model.predict(sample_array, batch_size=batch_size, verbose=0)

for i in output:
  pred_seq = []
  for step in range(predict_gap):
    pred_seq.append(CatFM_dic[np.argmax(i[step])])
  random_output.append(pred_seq)


In [ ]:
len(test_lst)

75

In [ ]:
random_output

[[460.0, 460.0, 460.0, 460.0, 460.0],
 [461.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [326.0, 326.0, 326.0, 326.0, 326.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 459.0, 459.0, 459.0, 459.0],
 [459.0, 459.0, 461.0, 461.0, 461.0],
 [460.0, 459.0, 459.0, 459.0, 459.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [459.0, 459.0, 459.0, 459.0, 459.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [459.0, 459.0, 459.0, 459.0, 459.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460.0, 326.0, 326.0, 326.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460.0, 460.0, 460.0, 460.0],
 [460.0, 460

In [ ]:
ac_lst = df_final[df_final['AC']==ac].sort_values('Date')['fm_cat'].tolist()

if len(ac_lst)*0.2 > 2*seq_length:
    test_lst = ac_lst[-int(len(ac_lst)*0.2):]
else: 
    test_lst = ac_lst[-(2*seq_length):]

sample = random.sample(test_lst,seq_length)


### without process detail AC pred_Score

In [ ]:
with open ('/content/drive/My Drive/Colab Notebooks/RR Folder/score_log.json', ) as file:
  score_log=json.load(file)

In [ ]:
acc_dic

{1: 1.0,
 2: 1.0,
 3: 0.0,
 4: 1.0,
 5: 1.0,
 6: 1.0,
 7: 0.5333333333333333,
 8: 1.0,
 9: 1.0,
 10: 1.0,
 11: 0.0,
 12: 1.0,
 13: 0.9333333333333333,
 14: 1.0,
 15: 0.13333333333333333,
 16: 1.0,
 17: 0.8666666666666667,
 18: 0.6666666666666666,
 19: 0.26666666666666666,
 22: 1.0,
 34: 1.0,
 35: 1.0}

In [ ]:
score_log['withoutProcess_AC_detail']=withoutProcess_AC_detail

In [ ]:
score_log

{'individual_ac': [{'1': 0.9333333333333333,
   '10': 1.0,
   '11': 0.0,
   '12': 1.0,
   '13': 1.0,
   '14': 1.0,
   '15': 1.0,
   '16': 1.0,
   '17': 1.0,
   '18': 0.8,
   '19': 1.0,
   '2': 1.0,
   '22': 0.8,
   '23': 1.0,
   '3': 0.0,
   '34': 1.0,
   '35': 1.0,
   '36': 1.0,
   '4': 1.0,
   '5': 1.0,
   '6': 1.0,
   '7': 0.8,
   '8': 1.0,
   '9': 1.0,
   'merge_11': 0.9333333333333333},
  {'1': 0.2,
   '10': 0.0,
   '11': 1.0,
   '12': 0.0,
   '13': 0.0,
   '14': 0.0,
   '15': 0.0,
   '16': 0.0,
   '17': 0.0,
   '18': 0.4,
   '19': 0.06666666666666667,
   '2': 0.0,
   '22': 0.2,
   '23': 0.0,
   '3': 1.0,
   '34': 0.0,
   '35': 0.0,
   '36': 0.0,
   '4': 0.0,
   '5': 0.0,
   '6': 0.0,
   '7': 0.2,
   '8': 0.0,
   '9': 0.0,
   'merge_11': 0.13333333333333333}],
 'overall_8_2': [0.9508771929824561, 0.07894736842105263],
 'withoutProcess_AC_detail': {1: 1.0,
  2: 1.0,
  3: 0.0,
  4: 1.0,
  5: 1.0,
  6: 1.0,
  7: 0.5333333333333333,
  8: 1.0,
  9: 1.0,
  10: 1.0,
  11: 0.0,
  12: 1.0,

# Function 

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 11 13:33:05 2020

@author: anyan.sun
"""
import pandas as pd
import numpy as np
import datetime
import random
from itertools import combinations
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, RepeatVector, Dense, TimeDistributed

from sklearn.model_selection import KFold
from keras.utils import to_categorical

def filter_col(df, col_lst, val_lst):
    '''
    col_lst: [the col that need to be filtered]
    val_lst: [the value of corresponding col to filter]
    '''
    
    for (col,val) in zip(col_lst, val_lst):
        df = df[df[col]==val]
    return df

# one hot encode sequence
def one_hot_encode(sequence, n_unique):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)

# so encoded idea is similar as getting the 'FM' dummies 
    # encoded = pd.get_dummies('FM')

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]


# convert encoded sequence to supervised learning
def to_supervised(encoded, n_in, n_out):
	# create lag copies of the sequence
	df = DataFrame(encoded)
	df = concat([df.shift(n_in-i-1) for i in range(n_in)], axis=1)
	# drop rows with missing values
	df.dropna(inplace=True)
	# specify columns for input and output pairs
	values = df.values
	width = encoded.shape[1]
	X = values.reshape(len(values), n_in, width)
	y = values[:, 0:(n_out*width)].reshape(len(values), n_out, width)
	return X, y


def df_resampling(df, date_col, fault_col, date_opt):
  error_dummies=pd.get_dummies(df[fault_col])
  error_dummies['datetime']=df[date_col].tolist()
  error_dummies['datetime']=pd.to_datetime(error_dummies['datetime'])
  error_dummies.index = error_dummies['datetime']
  df_dummies=error_dummies.resample(date_opt).sum()
  return df_dummies


def Get_Xlength(df_RR, date_range, date_col, duplicate='not_ignore'):
    '''
    This function is used to calculate the length of Xs input
        by studying about the all air carft
    date: count num of faults with in the date range
    duplicate: {ignore, not_ignore} 
        ignore: 460,460,460,459. all 460 counting as 1.\
                    so the total number of count is 2
        not_ignore: 460,460,460,459. all 460 counting as individual fault.\
                so the total number of count is 4
        
    
    '''
    
    fault_cnt=[0]
 
    for ac in df_RR['AC'].unique():
        
        df_filt = df_RR[(df_RR['AC']==ac) & (df_RR['Source Type']=='EEC') \
                        & (df_RR['Side']==1.)].sort_values(by=date_col).reset_index(drop=True)
        
        if duplicate=='not_ignore':
            df_filt1 = df_resampling(df_filt, 'Date', 'FM', str(date_range)+'D')
        
        else:
            df_filt1 = df_resampling(df_filt, 'Date', 'FM', str(date_range)+'D')
            df_filt1 = df_filt1.applymap(lambda x: False if x==0 else True)
    
        fault_cnt.extend([x for x in np.sum(df_filt1,1) if x!=0])
        
    fault_cnt.pop(0)
    
    return np.percentile(fault_cnt, 75)

def model_predict(n_in, n_out, encoded_length,\
                  batch_size, srcTR_encoded, tarTR_encoded, srcTE_encoded):
    # batch_size = len(X_train)

    model = Sequential() 
    model.add(LSTM(150, batch_input_shape=(batch_size, n_in, encoded_length), stateful=True))
    model.add(RepeatVector(n_out))
    model.add(LSTM(150, return_sequences=True, stateful=True))
    model.add(TimeDistributed(Dense(encoded_length, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(srcTR_encoded, tarTR_encoded, epochs=20, \
              batch_size=batch_size, verbose=5, shuffle=False)

    output = model.predict(srcTE_encoded, batch_size=batch_size, verbose=0)
    
    return output

def model_fit(n_in, n_out, encoded_length,\
                  batch_size, srcTR_encoded, tarTR_encoded):
    # batch_size = len(X_train)

    model = Sequential() 
    model.add(LSTM(150, batch_input_shape=(batch_size, n_in, encoded_length), stateful=True))
    model.add(RepeatVector(n_out))
    model.add(LSTM(150, return_sequences=True, stateful=True))
    model.add(TimeDistributed(Dense(encoded_length, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # optimizer= sgd/reul

    model.fit(srcTR_encoded, tarTR_encoded, epochs=20, \
              batch_size=batch_size, verbose=5, shuffle=False)

    return model
    


def generate_sequence(ac_lst, seq_length, predict_gap, rolling_step):  
  X_lst=[]
  y_lst=[]
  #drop_num = len(ac_lst) % seq_length
  #ac_lst_new = ac_lst[int(drop_num):]
  num_elements = len(ac_lst)
  for start in range(rolling_step, num_elements-seq_length-predict_gap-rolling_step, rolling_step):
      stop=start+seq_length
      X_lst.append(ac_lst[start:stop])
      y_lst.append(ac_lst[stop:stop+predict_gap])
  
  return X_lst, y_lst

## compare the predict result with the true values
def pred_score(output, y_test):
  detail_step=[]
  y_check=[]
  check_lst=[]
  for case in range(len(output)):
    indi_lst=[]
    y_lst=[]
    for step in range(predict_gap):
      indi_lst.append(np.argmax(output[case][step]))
      y_lst.append(np.argmax(y_test[case][step]))
    
    # print(case,step,np.argmax(tarTE_encoded[case][step]),np.argmax(output[case][step]))
      detail_step.append(np.argmax(y_test[case][step])==np.argmax(output[case][step]))
    # if able to predict more than one fualt type correctly without consider sequence
    check_lst.append(indi_lst)
    y_check.append(y_lst)

  Ture_Ture=[]
  False_True=[]
  for ind in range(len(check_lst)):
    # % fault predicted correctly among all faults happening in next 10 steps
    Ture_Ture.append(len(set(y_check[ind]) & set(check_lst[ind]))/len(set(y_check[ind])))
    # Fault that is predicted but not happening in the next 10 step
    False_True.append(len([fault for fault in set(check_lst[ind]) if fault not in set(y_check[ind])])/10)
    
  return  np.sum([1 for score in Ture_Ture if score!=0])/len(Ture_Ture), \
    np.sum([1 for score in False_True if score!=0])/len(False_True)
  
'''
  # train LSTM
for epoch in range(20):
	# generate new random sequence
	
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
	model.reset_states()
'''
        
#############################  



Using TensorFlow backend.


'\n  # train LSTM\nfor epoch in range(20):\n\t# generate new random sequence\n\t\n\t# fit model for one epoch on this sequence\n\tmodel.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)\n\tmodel.reset_states()\n'

In [ ]:
import pandas as pd
import numpy as np
import datetime
from itertools import combinations
import random


ntrain=int(len(df_ac)*0.8)
ntest=len(df_ac)-ntrain
n = 15
noffset=0
nelements=ntrain
sequence = ac_sequence
###

ncontext = n-3
xs, ys = [], []
noffset = max(0, noffset) # for debugging cases
nelements = min(nelements, len(sequence)) # ditto

 

for i in range(noffset, noffset + nelements - ncontext):
      try:
          x = sequence[i:i+ncontext]
          y = sequence[i+ncontext:i+ncontext+3]
          xs.append(x)
          ys.append(y)
      except:
          continue;
###

#xs = np.array(xs).reshape(-1,12,1) #np.reshape(np.array(xs), (-1, 1, 1)) #np.array(xs)
#ys = np.array(ys).reshape(-1,3,1) #np.reshape(np.array(ys), (-1, 1, 1)) #np.array(ys)

###
#xs_test = np.array(xs).reshape(-1,12,1) #
xs_test=np.reshape(np.array(xs), (-1, 1, 12)) #np.array(xs)
 
#ys_test = np.array(ys).reshape(-1,3,1) #np.reshape(np.array(ys), (-1, 1, 1)) #np.array(ys)

###
#xs_test = np.array(xs).reshape(-1,12,1) #
array(xs)
 
#ys_test = np.array(ys).reshape(-1,3,1) #np.reshape(np.array(ys), (-1, 1, 1)) #np.array(ys)
ys_test = np.array(ys).reshape(-1,1,3)
batch_size=12
model = Sequential()
model.add(LSTM(150, batch_input_shape=(batch_size,1,12), stateful=True))
model.add(RepeatVector(1))
model.add(LSTM(150, return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(3, activation='relu')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 

model.fit(xs_test, ys_test, epochs=20, \
             batch_size=12, verbose=5, shuffle=False)



Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Epoch 16/20
Epoch 17/20
Epoch 18/20
Epoch 19/20
Epoch 20/20


In [ ]:
model.predict(xs_test,batch_size=12,verbose=5)
#model.predict(np.array(testX))

array([[[1.9338676 , 1.969222  , 1.9985031 ]],

       [[1.9381351 , 2.08275   , 2.0833461 ]],

       [[2.2186139 , 2.132873  , 2.195899  ]],

       [[1.7870414 , 1.7115123 , 2.0189314 ]],

       [[2.0384457 , 2.1963577 , 1.9387312 ]],

       [[2.0441165 , 1.8100165 , 2.205     ]],

       [[1.8566127 , 2.2309582 , 1.9936111 ]],

       [[2.0953555 , 1.9280705 , 2.0076504 ]],

       [[2.1329308 , 2.233043  , 2.141828  ]],

       [[2.0635834 , 2.0056403 , 2.0733604 ]],

       [[2.137746  , 2.0075898 , 2.0788574 ]],

       [[2.0574508 , 1.912344  , 2.1379063 ]],

       [[1.71677   , 1.7227801 , 2.179648  ]],

       [[1.5953245 , 1.8550832 , 2.0379868 ]],

       [[1.5595528 , 1.4842763 , 2.5743933 ]],

       [[0.88195205, 2.1837997 , 1.3246647 ]],

       [[1.8810399 , 1.7780892 , 1.737543  ]],

       [[1.8486843 , 0.97372746, 2.7247925 ]],

       [[1.0145885 , 2.295031  , 1.9872121 ]],

       [[1.8379465 , 1.9145074 , 1.9184335 ]],

       [[2.0248413 , 2.0782819 , 2.16884

In [ ]:
model = Sequential() 
model.add(LSTM(150, batch_input_shape=(12, 12, 1), stateful=True))
model.add(RepeatVector(3))
model.add(LSTM(150, return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(1, activation='softmax')))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

 

model.fit(xs, ys, epochs=20, \
             batch_size=12, verbose=5, shuffle=False)

Epoch 1/20


InvalidArgumentError: ignored

In [ ]:
def gen_TrainTest(df_RR, TrainSplit_Cut, verti_cutMeth, rolling_step, AC_opt, cardinality,\
                  seq_length, predict_gap, batch_size):
    '''
    TrainSplit_Cut=='horizontal':
       Stacking 80% of length of each AC and taking all AC
    
    TrainSplit_Cut=='vertical':
       Stacking 80% AC but testing over another 20% AC 
       
    
    '''

    X_train=[]
    y_train=[]
    X_test=[]
    y_test=[]
    if TrainSplit_Cut=='horizontal':
        #for train, test in zip(train_ac[:5], test_ac[:5]):
        #train_ac should be list of ac to train, test_ac should be list of ac for testing
       
      train_ac = AC_opt[0:int(len(AC_opt)*0.8)]
      test_ac = AC_opt[int(len(AC_opt)*0.8):]
    
      df_Train = df_RR[df_RR['AC'].isin(train_ac)]
      df_Test = df_RR[df_RR['AC'].isin(test_ac)]
      

      for ac in df_Train['AC'].unique():
          ac_lst = df_Train[df_Train['AC']==ac].sort_values(['Date'])['fm_cat'].tolist()
          sub_X, sub_y = generate_sequence(ac_lst, seq_length, predict_gap, rolling_step)
          X_train.extend(sub_X)
          y_train.extend(sub_y)
          
      

      for ac in df_Test['AC'].unique():
        ac_lst = df_Test[df_Test['AC']==ac].sort_values('Date')['fm_cat'].tolist()
        sub_X, sub_y = generate_sequence(ac_lst, seq_length, predict_gap, rolling_step)
        X_test.extend(sub_X)
        y_test.extend(sub_y)
        
      X_train = to_categorical(X_train, num_classes=cardinality)
      y_train = to_categorical(y_train, num_classes=cardinality)
      X_train = X_train[len(X_train) % batch_size:]
      y_train = y_train[len(y_train) % batch_size:]
  
      X_test = to_categorical(X_test, num_classes=cardinality)
      y_test = to_categorical(y_test, num_classes=cardinality)
      X_test = X_test[len(X_test) % batch_size:]
      y_test = y_test[len(y_test) % batch_size:]
      

    elif TrainSplit_Cut=='vertical': 
      min_len = seq_length + batch_size
      df_train=pd.DataFrame(columns=df_RR['fm_cat'].unique())
      for ac in AC_opt:
          df_ac = df_RR[df_RR['AC']==ac].sort_values('Date')
          ac_lst=df_ac['fm_cat'].tolist()
          sub_X, sub_y = generate_sequence(ac_lst, seq_length, predict_gap, rolling_step)
          X_train.extend(sub_X)
          y_train.extend(sub_y)
          if verti_cutMeth=='20_per':
           
            if len(ac_lst)*0.2 > min_len+seq_length:
                test_lst=ac_lst[-int(len(ac_lst)*0.2):]
            else: 
                test_lst=ac_lst[-(min_len+seq_length):]
            sub_X, sub_y = generate_sequence(test_lst, seq_length, predict_gap, rolling_step)
            X_test.extend(sub_X)
            y_test.extend(sub_y)

          elif verti_cutMeth=='3_month':
            x=3
            while x<10:
              test_lst = df_ac[df_ac['Date']>datetime.date(df_ac['Date'].max().year, df_ac['Date'].max().month-x, df_ac['Date'].max().day)]['fm_cat'].tolist()
              if len(ac_lst)<min_len+seq_length:
                x=x+1

              else:
                x=11

            if len(ac_lst)==min_len+seq_length:
              test_lst=ac_lst
            else:
              test_lst=ac_lst[-(min_len+seq_length):]

            sub_X, sub_y = generate_sequence(test_lst, seq_length, predict_gap, rolling_step)
            X_test.extend(sub_X)
            y_test.extend(sub_y)
            
      X_train = to_categorical(X_train, num_classes=cardinality)
      y_train = to_categorical(y_train, num_classes=cardinality)
      X_train = X_train[len(X_train) % batch_size:]
      y_train = y_train[len(y_train) % batch_size:]
  
      X_test = to_categorical(X_test, num_classes=cardinality)
      y_test = to_categorical(y_test, num_classes=cardinality)
      X_test = X_test[len(X_test) % batch_size:]
      y_test = y_test[len(y_test) % batch_size:]
        
    return X_train, y_train, X_test, y_test